<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/04_PyTorch_Usage_Pratique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch : Usage Pratique pour le Médical

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Utiliser PyTorch pour analyser vos données médicales.

**Applications pratiques :**
- Calculs sur données patients
- Traitement d'images médicales
- Utilisation de modèles pré-entraînés
- Analyse par lots de dossiers

**Important:** Nous utilisons PyTorch comme outil, pas pour créer des modèles.

## Installation et Configuration

In [ ]:
# Installation de PyTorch et outils
!pip install torch torchvision matplotlib numpy pandas

import torch
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ GPU disponible: {torch.cuda.is_available()}")

## 📊 Exemple 1 : Calculs Médicaux avec PyTorch

**Cas pratique :** Calculer l'IMC de plusieurs patients en une fois.

In [ ]:
# Données de patients (taille en m, poids en kg)
patients_data = torch.tensor([
    [1.75, 70.0],  # Patient 1
    [1.68, 65.5],  # Patient 2
    [1.82, 85.0],  # Patient 3
    [1.60, 55.0],  # Patient 4
    [1.90, 95.0]   # Patient 5
])

print("👥 DONNÉES DES PATIENTS:")
print("Taille (m) | Poids (kg)")
print("-" * 20)
for i, (taille, poids) in enumerate(patients_data, 1):
    print(f"Patient {i}: {taille:.2f}m | {poids:.1f}kg")

In [ ]:
# Calcul vectorisé de l'IMC
taille = patients_data[:, 0]  # Première colonne
poids = patients_data[:, 1]   # Deuxième colonne

# IMC = poids / taille²
imc = poids / (taille ** 2)

print("📈 CALCUL DE L'IMC:")
print("=" * 30)
for i, imc_val in enumerate(imc, 1):
    # Classification IMC
    if imc_val < 18.5:
        categorie = "Insuffisance pondérale"
    elif imc_val < 25:
        categorie = "Normal"
    elif imc_val < 30:
        categorie = "Surpoids"
    else:
        categorie = "Obésité"
    
    print(f"Patient {i}: IMC = {imc_val:.1f} ({categorie})")

## 🖼️ Exemple 2 : Traitement d'Images Médicales

**Cas pratique :** Préparer des images médicales pour l'analyse.

In [ ]:
# Simuler une image médicale (radiographie)
np.random.seed(42)
image_simulee = np.random.rand(256, 256)

# Ajouter des structures médicales simulées
# Zone d'intérêt (par exemple, une lésion)
image_simulee[100:150, 120:170] += 0.3

# Convertir en tensor PyTorch
image_tensor = torch.from_numpy(image_simulee).float()
print(f"📏 Taille de l'image: {image_tensor.shape}")
print(f"📊 Valeurs min/max: {image_tensor.min():.3f} / {image_tensor.max():.3f}")

In [ ]:
# Transformations médicales courantes
def normaliser_image_medicale(image):
    """Normaliser une image médicale"""
    # Normalisation entre 0 et 1
    img_norm = (image - image.min()) / (image.max() - image.min())
    return img_norm

def ajuster_contraste(image, facteur=1.5):
    """Ajuster le contraste d'une image"""
    moyenne = image.mean()
    return moyenne + facteur * (image - moyenne)

# Appliquer les transformations
image_normalisee = normaliser_image_medicale(image_tensor)
image_contraste = ajuster_contraste(image_normalisee)

# Visualiser les résultats
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(image_tensor, cmap='gray')
plt.title('Image Originale')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(image_normalisee, cmap='gray')
plt.title('Image Normalisée')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(image_contraste, cmap='gray')
plt.title('Contraste Ajusté')
plt.colorbar()

plt.tight_layout()
plt.show()

print("✅ Transformations appliquées avec succès!")

## 🤖 Exemple 3 : Utiliser un Modèle Pré-entraîné

**Cas pratique :** Analyser des images avec un modèle existant.

In [ ]:
# Charger un modèle pré-entraîné (ResNet pour exemple)
modele = models.resnet18(pretrained=True)
modele.eval()  # Mode évaluation

print("🧠 Modèle ResNet18 chargé")
print(f"📊 Nombre de paramètres: {sum(p.numel() for p in modele.parameters()):,}")

In [ ]:
# Préparer une image pour le modèle
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Taille requise pour ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Simuler une image RGB (3 canaux)
image_rgb = torch.rand(3, 256, 256)

# Appliquer les transformations
image_preparee = transform(image_rgb)
image_batch = image_preparee.unsqueeze(0)  # Ajouter dimension batch

print(f"📏 Image originale: {image_rgb.shape}")
print(f"📏 Image préparée: {image_preparee.shape}")
print(f"📏 Batch pour modèle: {image_batch.shape}")

In [ ]:
# Analyser l'image avec le modèle
with torch.no_grad():
    predictions = modele(image_batch)

print(f"🎯 RÉSULTATS DE L'ANALYSE:")
print(f"Taille des prédictions: {predictions.shape}")
print(f"Confiance maximale: {predictions.max().item():.3f}")
print(f"Classe prédite: {predictions.argmax().item()}")

# Les 5 prédictions les plus probables
top5_prob, top5_idx = predictions.topk(5)
print(f"\n🏆 TOP 5 DES PRÉDICTIONS:")
for i, (prob, idx) in enumerate(zip(top5_prob[0], top5_idx[0])):
    print(f"  {i+1}. Classe {idx.item()}: {prob.item():.3f}")

## 📊 Exemple 4 : Analyse par Lots de Données

**Cas pratique :** Traiter plusieurs images médicales en une fois.

In [ ]:
# Créer un lot d'images médicales simulées
nb_images = 5
batch_images = torch.rand(nb_images, 1, 128, 128)  # 5 images en niveaux de gris

print(f"📦 LOT D'IMAGES CRÉÉ: {batch_images.shape}")
print(f"   - {nb_images} images")
print(f"   - 1 canal (niveaux de gris)")
print(f"   - 128x128 pixels")

# Ajouter des anomalies aléatoirement
for i in range(nb_images):
    if torch.rand(1) > 0.5:  # 50% de chance
        # Ajouter une "anomalie"
        x, y = torch.randint(20, 108, (2,))
        batch_images[i, 0, x:x+20, y:y+20] += 0.5
        print(f"  📍 Image {i+1}: anomalie ajoutée à ({x}, {y})")
    else:
        print(f"  ✅ Image {i+1}: normale")

In [ ]:
# Analyse statistique du lot
print("📈 ANALYSE STATISTIQUE DU LOT:")
print("=" * 40)

# Statistiques par image
for i in range(nb_images):
    img = batch_images[i, 0]
    moyenne = img.mean().item()
    ecart_type = img.std().item()
    valeur_max = img.max().item()
    
    print(f"📊 Image {i+1}:")
    print(f"   Moyenne: {moyenne:.3f}")
    print(f"   Écart-type: {ecart_type:.3f}")
    print(f"   Maximum: {valeur_max:.3f}")
    
    # Détection d'anomalie simple
    if valeur_max > 1.0:
        print(f"   🚨 ANOMALIE DÉTECTÉE (max > 1.0)")
    else:
        print(f"   ✅ Normal")
    print()

In [ ]:
# Visualiser tout le lot
fig, axes = plt.subplots(1, nb_images, figsize=(15, 3))

for i in range(nb_images):
    axes[i].imshow(batch_images[i, 0], cmap='gray')
    axes[i].set_title(f'Image {i+1}')
    axes[i].axis('off')
    
    # Marquer les anomalies
    if batch_images[i, 0].max() > 1.0:
        axes[i].set_title(f'Image {i+1} 🚨', color='red')

plt.suptitle('Lot d\'Images Médicales', fontsize=14)
plt.tight_layout()
plt.show()

## 🎯 Code Prêt à Utiliser

**Templates pour vos analyses médicales :**

In [ ]:
# TEMPLATE 1: Calcul d'IMC pour plusieurs patients
def calculer_imc_lot(tailles, poids):
    """
    Calculer l'IMC pour un groupe de patients
    
    Args:
        tailles: liste des tailles en mètres
        poids: liste des poids en kg
    
    Returns:
        tensor: IMC calculés
    """
    tailles_tensor = torch.tensor(tailles)
    poids_tensor = torch.tensor(poids)
    return poids_tensor / (tailles_tensor ** 2)

# TEMPLATE 2: Normaliser des images médicales
def preprocesser_images_medicales(images):
    """
    Préprocesser un lot d'images médicales
    
    Args:
        images: tensor d'images
    
    Returns:
        tensor: images normalisées
    """
    # Normalisation min-max
    images_norm = (images - images.min()) / (images.max() - images.min())
    return images_norm

# TEMPLATE 3: Détection d'anomalies simples
def detecter_anomalies(images, seuil=1.5):
    """
    Détecter des anomalies dans des images médicales
    
    Args:
        images: tensor d'images
        seuil: seuil de détection
    
    Returns:
        list: indices des images avec anomalies
    """
    anomalies = []
    for i, img in enumerate(images):
        if img.max() > seuil * img.mean():
            anomalies.append(i)
    return anomalies

print("💡 Templates PyTorch prêts pour vos analyses médicales!")

## ✅ Applications Pratiques

**Ce que vous pouvez faire avec PyTorch :**

1. **Calculs vectorisés** sur grandes cohortes de patients
2. **Préprocessing** d'images médicales par lots
3. **Utilisation de modèles** pré-entraînés existants
4. **Détection automatique** d'anomalies
5. **Analyse statistique** rapide de datasets

**Prochaine étape :** Nous verrons comment utiliser TorchXRayVision pour analyser spécifiquement des radiographies!